<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/Fine_tuningLLM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Imports and Initial Setup

In the initial cells, essential libraries and dependencies are imported, setting up the environment for the fine-tuning process. We use a custom trainer, configure GPU settings, and set up necessary libraries like Hugging Face's `transformers` and `peft` (Parameter-Efficient Fine-Tuning).

In [1]:
!pip install --upgrade peft transformers bitsandbytes datasets

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

**Load Datasets**

In [4]:
from datasets import load_dataset

# Load the Sanskrit dataset from OSCAR
dataset = load_dataset('oscar', 'unshuffled_deduplicated_sa', split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# Display a few samples

for i in range(3):
    print(f"Sample {i+1}:\n{dataset[i]['text']}\n")

Sample 1:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति । तस्‍य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्‍वयंबर इति चलचित्रं प्रकाश्यते ।
लट् लकार: एकवचनम् द्विवचनम्बहुवचनम्प्रथमपुरुष:गच्‍छतिगच्‍छत:गच्‍छन्तिमध्‍यमपुरुष:गच्‍छसिगच्‍छथ:गच्‍छथउत्‍तमपुरुष:गच्‍छामिगच्‍छाव:गच्‍छाम:

Sample 2:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति; अन्याः संस्थित्यः अपि सन्ति । अधिकं ज्ञातुम् अत्र उपयोगस्य संस्थितिं पश्यतु ।

Sample 3:
स्थिते च कवचे देहे नास्ति मृत्युश्च जीविनाम् । अस्त्रे-शस्त्रे-जले-वह्नौ सिद्धिश्चेन्नास्ति संशयः ॥ ४॥
प्राच्यां मां पातु भूतेशः आग्नेय्यां पातु शङ्करः । दक्षिणे पातु मां रुद्रो नैॠत्यां स्थाणुरेव च ॥ ११॥



**Data Cleaning**

In [6]:
import re

def clean_text(example):
    text = example['text']
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    return {'text': text}

# Apply the cleaning function
dataset = dataset.map(clean_text)

In [7]:
def filter_short_texts(example):
    return len(example['text'].split()) > 10  # Keep texts longer than 10 words

dataset = dataset.filter(filter_short_texts)

In [8]:
dataset[0]

{'id': 0,
 'text': 'अनिरुद्धनगरे क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । तस्य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्वयंबर इति चलचित्रं प्रकाश्यते । लट् लकार एकवचनम् द्विवचनम्बहुवचनम्प्रथमपुरुषगच्छतिगच्छतगच्छन्तिमध्यमपुरुषगच्छसिगच्छथगच्छथउत्तमपुरुषगच्छामिगच्छावगच्छाम'}

In [9]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/hf_token.txt', 'r') as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(token=hf_token)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
model_name = 'google/gemma-2-2b-it'


In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Tokenize the first example in the dataset
tokens = tokenizer(dataset[0]['text'])

# Get the length of the tokenized sentence
length = len(tokens['input_ids'])

# Decode the tokens back to the original sentence
decoded_sentence = tokenizer.decode(tokens['input_ids'])

print(f"Sentence: {decoded_sentence}")
print(f"Length (in tokens): {length}")


Sentence: <bos>अनिरुद्धनगरे क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । तस्य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्वयंबर इति चलचित्रं प्रकाश्यते । लट् लकार एकवचनम् द्विवचनम्बहुवचनम्प्रथमपुरुषगच्छतिगच्छतगच्छन्तिमध्यमपुरुषगच्छसिगच्छथगच्छथउत्तमपुरुषगच्छामिगच्छावगच्छाम
Length (in tokens): 149


In [11]:
dataset[2]

{'id': 2,
 'text': 'स्थिते च कवचे देहे नास्ति मृत्युश्च जीविनाम् । अस्त्रेशस्त्रेजलेवह्नौ सिद्धिश्चेन्नास्ति संशयः ॥ ४॥ प्राच्यां मां पातु भूतेशः आग्नेय्यां पातु शङ्करः । दक्षिणे पातु मां रुद्रो नैॠत्यां स्थाणुरेव च ॥ ११॥'}

In [12]:
# Tokenize the first example in the dataset
tokens = tokenizer(dataset[2]['text'])

# Get the length of the tokenized sentence
length = len(tokens['input_ids'])

# Decode the tokens back to the original sentence
decoded_sentence = tokenizer.decode(tokens['input_ids'])

print(f"Sentence: {decoded_sentence}")
print(f"Length (in tokens): {length}")


Sentence: <bos>स्थिते च कवचे देहे नास्ति मृत्युश्च जीविनाम् । अस्त्रेशस्त्रेजलेवह्नौ सिद्धिश्चेन्नास्ति संशयः ॥ ४॥ प्राच्यां मां पातु भूतेशः आग्नेय्यां पातु शङ्करः । दक्षिणे पातु मां रुद्रो नैॠत्यां स्थाणुरेव च ॥ ११॥
Length (in tokens): 105


In [13]:
# Initialize variables to store the sentence with the maximum length
max_length = 0
max_sentence = ""

# Iterate through the dataset
for example in dataset:
    tokens = tokenizer(example['text'])
    length = len(tokens['input_ids'])

    if length > max_length:
        max_length = length
        max_sentence = example['text']

# Output the sentence with the maximum length
print(f"Sentence with maximum length:\n{max_sentence}")
print(f"Token length: {max_length}")



Sentence with maximum length:
ब्रह्म॒ सन्ध॑त्तं॒ तन्मे॑ जिन्वतम् । क्ष॒त्त्र सन्ध॑त्तं॒ तन्मे॑ जिन्वतम् । इष॒॒ सन्ध॑त्तं॒ तां मे॑ जिन्वतम् । ऊर्ज॒॒ सन्ध॑त्तं॒ तां मे॑ जिन्वतम् । र॒यि सन्ध॑त्तं॒ तां मे॑ जिन्वतम् । पुष्टि॒॒ सन्ध॑त्तं॒ तां मे॑ जिन्वतम् । प्र॒जा सन्ध॑त्तं॒ तां मे॑ जिन्वतम् । प॒शून्थ्सन्ध॑त्तं॒ तान्मे॑ जिन्वतम् । स्तु॒तो॑ऽसि॒ जन॑धाः । दे॒वास्त्वा॑ शुक्र॒पा प्रण॑यन्तु ।।  ।। प्रा॒णं य॒ज्ञप॑तये धत्तम् । चक्षु॑स्स्थ॒श्चक्षु॑र्मे धत्तम् । चक्षु॑र्य॒ज्ञाय॑ धत्तम् । चक्षु॑र्य॒ज्ञप॑तये धत्तम् । श्रोत्र॑ स्थ॒श्श्रोत्रं॑ मे धत्तम् । श्रोत्रं॑ य॒ज्ञाय॑ धत्तम् । श्रोत्रं॑ य॒ज्ञप॑तये धत्तम् । तौ दे॑वौ शुक्रामन्थिनौ । क॒ल्पय॑तं॒ दैवी॒र्विशः॑ । क॒ल्पय॑तं॒ मानु॑षीः ।।  ।। इष॒मूर्ज॑म॒स्मासु॑ धत्तम् । प्रा॒णान्प॒शुषु॑ । प्र॒जां मयि॑ च॒ यज॑माने च । निर॑स्त॒श्शण्डः॑ । निर॑स्तो॒ मर्कः॑ । अप॑नुत्तौ॒ शण्डा॒मर्कौ॑ स॒हामुना । शु॒क्रस्य॑ स॒मिद॑सि । म॒न्थिन॑स्स॒मिद॑सि । स प्र॑थ॒मस्संकृ॑तिर्वि॒श्वक॑र्मा । स प्र॑थ॒मो मि॒त्रो वरु॑णो अ॒ग्निः । स प्र॑थ॒मो बृह॒स्पति॑श्चिकि॒त्वान् । तस्मा॒ इन्द्रा॑य सु॒तमा

**Load Model**

**Check whether Cuda is available**

In [14]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**LORA Configurations**

In [15]:
from transformers import BitsAndBytesConfig

#bnb_config = BitsAndBytesConfig(load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.CAUSAL_LM,
)

In [17]:
model = get_peft_model(model, lora_config)

In [18]:
for name, param in model.named_parameters():
    if 'lora' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

**Number of trainable parameters**

In [19]:
# Verify trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} | All params: {all_params} | "
        f"Trainable percentage: {100 * trainable_params / all_params:.2f}%"
    )

print_trainable_parameters(model)

Trainable params: 6389760 | All params: 1608593664 | Trainable percentage: 0.40%


In [20]:
#model.gradient_checkpointing_enable()

**Prepare Tokenized Dataset**

In [21]:
# Prepare dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256,
    )
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
print(tokenized_dataset[0])  # Check the first example in the tokenized dataset

{'id': 0, 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 236088, 235530, 42894, 235844, 54736, 235530, 235827, 36352, 2280, 40831, 236127, 117062, 196908, 33920, 17285, 124267, 54701, 22870, 18852, 46577, 235485, 9043, 8104, 45884, 235527, 21640, 10321, 124199, 11201, 56667, 236062, 18029, 213831, 235437, 153743, 161306, 60152, 235699, 172684, 117062, 56667, 3640, 86532, 235527, 21640, 52348, 237453, 118948, 96061, 5444, 237453, 27456, 235527, 172684, 13568, 237453, 21640, 10321, 22740, 212361, 18029, 12255, 235699, 82046, 60152, 236396, 12255, 235550, 35871, 65892, 9043, 117232, 235643, 104371, 14176, 22870, 118948, 96061, 5444, 235563, 12218, 22729, 87046, 18249, 21640, 8460, 235935, 235485, 8460, 

**Customized Data Collator**

In [22]:
# Custom data collator
def custom_data_collator(features):
    batch = {}
    for key in features[0].keys():
        batch[key] = torch.stack([torch.tensor(f[key]) for f in features]).to('cuda')
    return batch

# Custom Trainer Initialization

A `CustomTrainer` instance is initialized with the model, training arguments, and dataset. This custom trainer coordinates the training process, using the arguments and model components configured earlier.

In [23]:
class CustomTrainer(Trainer):
    def prepare_inputs(self, inputs):
        # Ensure inputs are on GPU
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        return inputs

In [24]:
# Get a batch of data
batch = tokenized_dataset[:2]  # Take the first 2 samples

# Convert batch to tensors and move to GPU
inputs = {
    'input_ids': torch.tensor(batch['input_ids']).to('cuda'),
    'attention_mask': torch.tensor(batch['attention_mask']).to('cuda'),
    'labels': torch.tensor(batch['labels']).to('cuda'),
}

**Set model to evaluation mode**

In [25]:
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    loss = outputs.loss
    print(f"Loss: {loss.item()}")
    print(f"Loss requires grad: {loss.requires_grad}")

Loss: 20.809642791748047
Loss requires grad: False


**Set model to training mode**

In [26]:
model.train()

outputs = model(**inputs)
loss = outputs.loss

print(f"Loss: {loss.item()}")
print(f"Loss requires grad: {loss.requires_grad}")

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Loss: 20.809642791748047
Loss requires grad: True


In [27]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: requires_grad={param.requires_grad}, device={param.device}")

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_pr

In [28]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Setting Training Arguments

Training arguments are configured here, including batch size, number of training epochs, learning rate, and other key settings. This configuration is essential for controlling the fine-tuning process.

In [29]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/results/',
    num_train_epochs=2,
    per_device_train_batch_size=5,  # Adjust based on GPU memory
    gradient_accumulation_steps=4,  # Adjust to maintain effective batch size
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=1,
    save_steps=5000,
    gradient_checkpointing=False,
    optim='adamw_bnb_8bit',
    dataloader_pin_memory=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=custom_data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# Memory Management and Training Execution

To manage memory effectively on the GPU, we clear the CUDA cache before starting the actual training. Then, the `trainer.train()` command is used to start the fine-tuning.

The training progress, including the number of steps completed, current loss, and other details, is displayed to monitor the model's learning curve.

In [31]:
# Clear cache and start training
torch.cuda.empty_cache()

In [32]:
trainer.train()

Step,Training Loss
500,2.296700


TrainOutput(global_step=668, training_loss=2.24030322229077, metrics={'train_runtime': 3235.0584, 'train_samples_per_second': 4.129, 'train_steps_per_second': 0.206, 'total_flos': 4.166369671171277e+16, 'train_loss': 2.24030322229077, 'epoch': 2.0})

# Saving the Trained Model

Once training is complete, we save the fine-tuned model and tokenizer to a directory for future usage in generating Sanskrit text.


In [33]:
trainer.save_model('fine-tuned-gemma2-sanskrit-lora')
tokenizer.save_pretrained('fine-tuned-gemma2-sanskrit-lora')

('fine-tuned-gemma2-sanskrit-lora/tokenizer_config.json',
 'fine-tuned-gemma2-sanskrit-lora/special_tokens_map.json',
 'fine-tuned-gemma2-sanskrit-lora/tokenizer.model',
 'fine-tuned-gemma2-sanskrit-lora/added_tokens.json',
 'fine-tuned-gemma2-sanskrit-lora/tokenizer.json')

# Model Loading and Evaluation Setup

In this section, we load the fine-tuned model into evaluation mode for generating Sanskrit text. The `PeftModel` allows efficient loading and evaluation.

In [6]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import os, gc
offload_dir = "./offload"
os.makedirs(offload_dir, exist_ok=True)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="sequential",  # 👈️ Avoid RAM peak
    offload_folder=offload_dir  # Enables offloading to the CPU if needed
)

model = PeftModel.from_pretrained(model, 'fine-tuned-gemma2-sanskrit-lora')
gc.collect()
# Set the model to evaluation mode
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
        

# Text Generation Function

A function, `generate_text`, is defined for generating Sanskrit text based on a provided prompt. Parameters for temperature, top-p sampling, and maximum length control the creativity and diversity of the generated text.


In [9]:
def generate_text(prompt, max_length=128, num_return_sequences=1):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Generate output sequences
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,           # Use sampling for more diverse outputs
            temperature=0.7,          # Adjust temperature for creativity
            top_p=0.9,                # Use top-p sampling
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the generated tokens
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

# Example Prompt and Generated Text

Using the `generate_text` function, we provide an example Sanskrit prompt and generate text based on it to showcase the model's capability to create coherent Sanskrit sentences.

In [11]:
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-gemma2-sanskrit-lora")

prompt = "पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति"
generated_texts = generate_text(prompt, max_length=256)
print("Generated Text:")
print(generated_texts[0])

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Generated Text:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति । अत्र प्रथमविषयः एव कर्तृवार्तायाः प्रथमभागः भवति । अस्मिन् विषये एव चरित्रभाषणः च भवति । चरित्रभाषणस्य उद्देश्यः एव चरित्राणां प्रोत्साहनः । अत्र एव पञ्चमविषयः एव सञ्चितानां कल्पनात्मकवर्णनानां अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति । अत्र प्रथमविषयः एव पञ्चमविषयस्य प्रारम्भिकभागः भवति । अस्मिन् विषये एव पञ्चमविषयस्य उद्देश्यः एव कल्पनात्मकवर्णनानां प्रोत्साहनः । अत्र एव आठमविषयः एव सञ्च


In [12]:
prompt = "अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति ।"
generated_texts = generate_text(prompt, max_length=256)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति । रामलीला सम्‍प्रतिं गौरववर्णं चतुर्विंशति अस्ति । अनिरुद्धनगरस्य नागरिकस्य सत्कारं चतुर्विंशति अस्ति । एवम् अभ्युत्पादयति नागरिकं नागरिकस्य सत्कारं चतुर्विंशति अस्ति । अनिरुद्धनगरस्य प्रसिद्धं नागरिकं रामलीला सम्‍प्रतियाम् आसीत् । एवम् आसीत् । अनिरुद्धनगरस्य रामलीला सम्‍प्रतियाम् आसीत् । एवम् आसीत् । अनिरुद्धनगरस्य रामलीला सम्‍प्रतियाम् आसीत् । एवम् आसीत् । अनिरुद्धनगरस्य रामलीला सम्‍प्रतियाम् आसीत् । एवम् आसीत् । अनिरुद्धनगरस्य रामलीला सम्‍प्रतियाम् आसीत् । एवम् आसीत् ।


In [13]:

prompt = "सर्वे भवन्तु सुखिनः, सर्वे सन्तु निरामयाः।"
generated_texts = generate_text(prompt, max_length=256)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
सर्वे भवन्तु सुखिनः, सर्वे सन्तु निरामयाः। सर्वे भवन्तु सन्तु निरम्रायः। ॐ शान्तिः शान्तिः शान्तिः। ॐ।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।


In [14]:

prompt = "कर्मण्येवाधिकारस्ते मा फलेषु कदाचन मा कर्मफलहेतुर्भूर्मा ते सङ्गोऽस्त्वकर्मणि।"
generated_texts = generate_text(prompt, max_length=256)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
कर्मण्येवाधिकारस्ते मा फलेषु कदाचन मा कर्मफलहेतुर्भूर्मा ते सङ्गोऽस्त्वकर्मणि। अस्माकर्मणाङ्गस्ते तमं कर्मफलं भूयोरुच्यते कर्मफलं न भूयोरुच्यते॥ इति कर्मणिभूयोरुच्यते ॥ इति न कर्मणिभूयोरुच्यते ॥ इति एव कर्मणिभूयोरुच्यते ॥ इति एव न कर्मणिभूयोरुच्यते ॥ इति कर्मणिभूयोरुच्यते ॥ इति एव न कर्मणिभूयोरुच्यते ॥ इति कर्मणिभूयोरुच्यते ॥ इति एव न कर्मणिभूयोरुच्यते ॥ इति कर्मणिभूयोरुच्यते ॥ इति एव न कर्मणिभूयोरुच्यते ॥ इति कर्मणिभूयोरुच्यते ॥ इति एव न कर्म
